# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,0.81,93,75,3.09,AR,1711763104
1,1,coquimbo,-29.9533,-71.3436,15.85,94,100,2.06,CL,1711762670
2,2,ust'-ilimsk,58.0006,102.6619,-11.42,81,100,4.02,RU,1711763326
3,3,tiksi,71.6872,128.8694,-23.73,89,100,4.58,RU,1711763211
4,4,ancud,-41.8697,-73.8203,10.45,82,37,2.86,CL,1711763226


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
map=city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    c="City",
    size="Humidity",
    frame_height=550
)
# Display the map
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
temp_df=city_data_df.loc[(city_data_df["Max Temp"]>21) & (city_data_df["Max Temp"]<27)]
wind_df=temp_df.loc[temp_df["Wind Speed"]<4.5]
narrow_df=wind_df.loc[wind_df["Cloudiness"]==0]

# Drop any rows with null values
narrow_df.dropna()

# Display sample data
narrow_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,37,mount gambier,-37.8333,140.7667,25.79,32,0,1.75,AU,1711763376
56,56,saint-pierre,-21.3393,55.4781,25.82,73,0,1.54,RE,1711763405
128,128,karachi,24.9056,67.0822,22.90,78,0,3.09,PK,1711763403
139,139,san luis,-33.2950,-66.3356,22.50,53,0,2.87,AR,1711763534
161,161,duba,27.3513,35.6901,23.21,53,0,1.67,SA,1711763568
216,216,hasaki,35.7333,140.8333,22.94,60,0,3.34,JP,1711763654
217,217,codrington,-38.2667,141.9667,23.06,42,0,3.82,AU,1711763656
423,423,pasni,25.2631,63.4710,22.58,69,0,2.64,PK,1711763962
435,435,bell ville,-32.6259,-62.6887,21.48,57,0,3.64,AR,1711763980
478,478,springbok,-29.6643,17.8865,21.09,35,0,2.60,ZA,1711764051


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=narrow_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
37,mount gambier,AU,-37.8333,140.7667,32,
56,saint-pierre,RE,-21.3393,55.4781,73,
128,karachi,PK,24.9056,67.0822,78,
139,san luis,AR,-33.2950,-66.3356,53,
161,duba,SA,27.3513,35.6901,53,
216,hasaki,JP,35.7333,140.8333,60,
217,codrington,AU,-38.2667,141.9667,42,
423,pasni,PK,25.2631,63.4710,69,
435,bell ville,AR,-32.6259,-62.6887,57,
478,springbok,ZA,-29.6643,17.8865,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit":20,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=hotel_df.loc[index,"Lat"]
    longitude=hotel_df.loc[index,"Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    params['categories']=f"accommodation.hotel"


    # Make and API request using the params dictionaty

    name_address = requests.get(base_url,params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mount gambier - nearest hotel: The Old Mount Gambier Gaol
saint-pierre - nearest hotel: Tropic Hotel
karachi - nearest hotel: Quetta Agha Hotel
san luis - nearest hotel: Hotel Dos Venados
duba - nearest hotel: No hotel found
hasaki - nearest hotel: Choshi Plaza Hotel
codrington - nearest hotel: No hotel found
pasni - nearest hotel: Juddi Hotel
bell ville - nearest hotel: No hotel found
springbok - nearest hotel: Elkoweru Guesthouse


,City,Country,Lat,Lng,Humidity,Hotel Name
37,mount gambier,AU,-37.8333,140.7667,32,The Old Mount Gambier Gaol
56,saint-pierre,RE,-21.3393,55.4781,73,Tropic Hotel
128,karachi,PK,24.9056,67.0822,78,Quetta Agha Hotel
139,san luis,AR,-33.2950,-66.3356,53,Hotel Dos Venados
161,duba,SA,27.3513,35.6901,53,No hotel found
216,hasaki,JP,35.7333,140.8333,60,Choshi Plaza Hotel
217,codrington,AU,-38.2667,141.9667,42,No hotel found
423,pasni,PK,25.2631,63.4710,69,Juddi Hotel
435,bell ville,AR,-32.6259,-62.6887,57,No hotel found
478,springbok,ZA,-29.6643,17.8865,35,Elkoweru Guesthouse


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map=hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    c="City",
    size="Humidity",
    frame_height=550,
    hover_cols=["Lng","Lat","City","Humidity","Hotel Name","Country"]
    
    
)
# Display the map
map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)